In [2]:
wd = '/mnt/openfact/users/msawinski/factue-task2'
import sys, os
os.chdir(wd)

In [5]:
import pandas as pd
from pathlib import Path
import os
root = Path("data/llm_output/persuasion")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)

In [7]:
df['source_file'].value_counts().sort_index()

source_file
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-pl/batch_0000.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-pl/batch_0001.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-pl/batch_0002.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-pl/batch_0003.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-pl/batch_0004.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-pl/batch_0005.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justification/Appeal_to_Values_v001/train/train-pl/batch_0006.parquet    20
data/llm_output/persuasion/detect/ModelName.LLAMA_31_8B-justific

In [8]:
df.head(3)

,filename,start,end,text_lang,text,label_bin,labels_multi,identifier,technique_id,raw_result,Description,Verdict,pred,gold,source_file
0,pl_abortion_11_04_2024_n04.txt,0,95,PL,Wicemarszałek Włodzimierz Czarzasty:\nDziękuję...,False,[],ModelName.LLAMA_31_8B-justification/Appeal_to_...,None,"[{\n ""Description"": null,\n ""Verdict"": false...","[None, ]","[False, False]",False,False,data/llm_output/persuasion/detect/ModelName.LL...
1,pl_abortion_11_04_2024_n04.txt,97,861,PL,Poseł Katarzyna Ueberhan:\nSzanowny Panie Mars...,True,"[Conversation_Killer, Exaggeration-Minimisatio...",ModelName.LLAMA_31_8B-justification/Appeal_to_...,None,"[{\n ""Description"": ""Appeal to Values techniq...",[Appeal to Values technique is used by evoking...,"[True, True, True, False, True]",True,False,data/llm_output/persuasion/detect/ModelName.LL...
2,pl_abortion_11_04_2024_n04.txt,863,1318,PL,Aborcję farmakologiczną można zastosować\nzara...,True,"[Conversation_Killer, Consequential_Oversimpli...",ModelName.LLAMA_31_8B-justification/Appeal_to_...,None,"[{\n ""Description"": ""Appeal to Values techniq...",[Appeal to Values technique was used by mentio...,"[True, True, True, True, True]",True,False,data/llm_output/persuasion/detect/ModelName.LL...


In [10]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

# Metrics
acc = accuracy_score(df['gold'], df['pred'])
recall = recall_score(df['gold'], df['pred'], average='binary')
f1 = f1_score(df['gold'], df['pred'], average='binary')

print(f"Accuracy: {acc:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")

Accuracy: 0.685
Recall: 0.957
F1-score: 0.583
